<a href="https://colab.research.google.com/github/ashishkathane599/E-Waste-Generation-classification-/blob/main/E_Waste_Generation_Classificaton.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

import requrired libraries

In [5]:
# Install the gradio library
!pip install gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.6/323.6 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.3/95.3 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 98.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 4.5 MB/s eta 0:00:00


In [6]:
import tensorflow as tf
from tensorflow.keras import layers,models,optimizers,callbacks
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.applications import EfficientNetV2B0
from tensorflow.keras.applications.efficientnet import preprocess_input
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix,classification_report
import gradio as gr
from PIL import Image

In [7]:
testpath= r'/content/drive/MyDrive/E-Waste classification dataset/modified-dataset/test'
trainpath= r"/content/drive/MyDrive/E-Waste classification dataset/modified-dataset/train"
validpath = r"/content/drive/MyDrive/E-Waste classification dataset/modified-dataset/val"

In [8]:
datatrain = tf.keras.utils.image_dataset_from_directory(trainpath,shuffle=True,image_size=(128,128),batch_size=32,validation_split=False)

Found 1181 files belonging to 10 classes.
